In [8]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import src.Fluid_CP as FCP
import src.Compressormodel_SP as CSP

In [ ]:
# parameters

## cooking pasta

Tw_morning = 20 #degree Celsius

Tp_before = 7 #degree celsius

P_batch = 2 # kg --> 0.5 kg per basket

tc = 7 #minutes --> pasta is cooked for 7 minutes per batch

Tp_after = 85 #degree celsius

tb = 5 #minutes --> time between cooking batches

Tw_op_min = 85 #degree celsius --> minimal temperature during cooking

Tw_op_max = 93 #degree celsius --> maximal temperature during cooking

## other




